In [1]:
import random
def generate_theme_title_and_story(campaign_title: str, insight: str) -> tuple[str, str]:
    title_templates = [
        "The Power of {keyword}",
        "Mastering {keyword} for Growth",
        "Why {keyword} Matters Now",
        "{keyword}: A New Approach",
        "Unlocking {keyword}"
    ]

    insight_keywords = insight.split()
    keyword = random.choice(insight_keywords if insight_keywords else ["change"])
    title = random.choice(title_templates).format(keyword=keyword.capitalize())

    story = f"This theme explores how '{keyword}' relates to {campaign_title.lower()}. We’ll break down why this is crucial and how to apply it practically."
    return title, story
generate_theme_title_and_story("123", "123")

('123: A New Approach',
 "This theme explores how '123' relates to 123. We’ll break down why this is crucial and how to apply it practically.")

In [ ]:
import asyncio


async def generate_topic():
    response = await openai.ChatCompletion.acreate(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a creative assistant."},
            {"role": "user", "content": "Generate a unique topic for a technology blog."}
        ]
    )
    topic = response.choices[0].message['content']
    print(f"Generated Topic: {topic}")

# Run the asynchronous function
asyncio.run(generate_topic())

In [ ]:
from openai import OpenAI
import asyncio
from dotenv import load_dotenv
import nest_asyncio
import pandas as pd
import time
import os

# Load environment variables and apply nest_asyncio
load_dotenv()
nest_asyncio.apply()

# Initialize OpenAI client with DeepSeek endpoint
client = OpenAI(api_key=os.getenv("DEEPSEEK_API_KEY"), base_url="https://api.deepseek.com")

# Define the topics for the posts
topics = [
    "Benefits of meditation",
    "How to start programming",
    "Climate change solutions",
    "Healthy meal prep ideas",
    "Financial planning tips",
    "Productivity hacks",
    "Travel on a budget",
    "Home workout routines",
    "Book recommendations for beginners",
    "Tech trends 2025"
]

async def generate_post(topic):
    """Generate a post about a specific topic using DeepSeek."""
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes blog posts"},
                {"role": "user", "content": f"Write a short blog post about {topic}. Keep it under 300 words."},
            ],
            stream=False
        )
        content = response.choices[0].message.content
        
        print(f"Generated post for: {topic}")
        
        return {
            "topic": topic,
            "content": content,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
    except Exception as e:
        print(f"Error generating post for {topic}: {e}")
        return {
            "topic": topic,
            "content": f"Error: {str(e)}",
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }

async def process_topics():
    """Process all topics concurrently."""
    tasks = []
    for topic in topics:
        task = asyncio.create_task(generate_post(topic))
        tasks.append(task)
    
    results = await asyncio.gather(*tasks)
    return results

async def main():
    print("Generating 10 posts asynchronously with DeepSeek...")
    start_time = time.time()
    
    results = await process_topics()
    
    # Create DataFrame from results
    df = pd.DataFrame(results)
    
    # Display summary of the DataFrame
    print("\nGenerated posts summary:")
    for index, row in df.iterrows():
        print(f"{index+1}. {row['topic']} - {len(row['content'])} chars")
    
    # Export to Excel
    excel_filename = "deepseek_generated_posts.xlsx"
    df.to_excel(excel_filename, index=False)
    
    end_time = time.time()
    print(f"\nExported {len(results)} posts to {excel_filename}")
    print(f"Total time: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    asyncio.run(main())

Generating 10 posts asynchronously with DeepSeek...
Generated post for: Benefits of meditation
Generated post for: How to start programming
Generated post for: Climate change solutions
Generated post for: Healthy meal prep ideas
Generated post for: Financial planning tips
Generated post for: Productivity hacks
Generated post for: Travel on a budget
Generated post for: Home workout routines
Generated post for: Book recommendations for beginners
Generated post for: Tech trends 2025

Generated posts summary:
1. Benefits of meditation - 1359 chars
2. How to start programming - 1472 chars
3. Climate change solutions - 1494 chars
4. Healthy meal prep ideas - 1511 chars
5. Financial planning tips - 1394 chars
6. Productivity hacks - 1185 chars
7. Travel on a budget - 1454 chars
8. Home workout routines - 1395 chars
9. Book recommendations for beginners - 1418 chars
10. Tech trends 2025 - 1705 chars


ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
import os
import json
import asyncio
import time
import pandas as pd
from dotenv import load_dotenv
import nest_asyncio
from google import genai
from google.genai import types
from typing import List, Dict, Any
from pydantic import BaseModel, ValidationError

# Load environment variables and apply nest_asyncio
load_dotenv()
nest_asyncio.apply()

# Define topics for generation
topics = [
    "Benefits of meditation",
    "How to start programming",
    "Climate change solutions",
    "Healthy meal prep ideas",
    "Financial planning tips",
    "Productivity hacks",
    "Travel on a budget",
    "Home workout routines",
    "Book recommendations for beginners",
    "Tech trends 2025"
]


# System prompt - adapt this to your needs
system_prompt = """
You are an assistant that writes engaging blog posts. 
Create content that is informative, well-structured, and under 300 words.
Format your response as a JSON object with 'title' and 'content' fields.
"""

class BlogPost(BaseModel):
    title: str
    content: str

async def generate_post_with_gemini(topic: str) -> Dict[str, Any]:
    """Generate a post about a specific topic using Google Gemini."""
    try:
        # Initialize the client similar to your implementation
        client = genai.Client(api_key=os.getenv("GEMINI_API_KEY") )
        
        # Create the prompt
        prompt = f"Write a short blog post about {topic}. Keep it under 300 words."
        
        # Generate response using Gemini API - following your implementation pattern
        response = client.models.generate_content(
            model='gemini-2.0-flash',
            contents=prompt,
            config={
                'response_mime_type': 'application/json',
                'response_schema': BlogPost,
                'system_instruction': types.Part.from_text(text=system_prompt),
            },
        )
        
        # Extract the response - similar to your approach
        print(f"Generated post for: {topic}")
        content = json.loads(response.text)
        blog_post = BlogPost(**content)
        
        return {
            "topic": topic,
            "title": blog_post.title,
            "content": blog_post.content,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Gemini API Error: {str(e)}")


# This function handles creating the semaphore and processing tasks
async def process_with_semaphore(topics: List[str], concurrency: int = 10):
    """Process topics with a semaphore to limit concurrent API calls."""
    # Use a semaphore to control concurrency
    semaphore = asyncio.Semaphore(concurrency)
    
    async def bounded_generate(topic):
        async with semaphore:
            return await generate_post_with_gemini(topic)
    
    tasks = [bounded_generate(topic) for topic in topics]
    return await asyncio.gather(*tasks)

async def main():
    print("Generating posts asynchronously with Google Gemini...")
    start_time = time.time()
    
    # Process all topics with controlled concurrency
    results = await process_with_semaphore(topics)
    
    # Create a DataFrame from the results
    df = pd.DataFrame(results)
    
    # Display summary of the DataFrame
    print("\nGenerated posts summary:")
    for index, row in df.iterrows():
        print(f"{index+1}. {row['title']} - {len(row['content'])} chars")
    
    # Export to Excel
    excel_filename = "gemini_generated_posts.xlsx"
    df.to_excel(excel_filename, index=False)
    
    end_time = time.time()
    print(f"\nExported {len(results)} posts to {excel_filename}")
    print(f"Total time: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    asyncio.run(main())

Generating posts asynchronously with Google Gemini...
Generated post for: Benefits of meditation
Generated post for: How to start programming
Generated post for: Climate change solutions
Generated post for: Healthy meal prep ideas
Generated post for: Financial planning tips
Generated post for: Productivity hacks
Generated post for: Travel on a budget
Generated post for: Home workout routines
Generated post for: Book recommendations for beginners
Generated post for: Tech trends 2025

Generated posts summary:
1. Unlock Inner Peace: The Benefits of Meditation - 1348 chars
2. Embark on Your Coding Journey: A Beginner's Guide - 1076 chars
3. Fighting Back: Practical Climate Change Solutions - 1221 chars
4. Fuel Your Week: Easy & Healthy Meal Prep Ideas - 1209 chars
5. Smart Financial Planning Tips for a Secure Future - 791 chars
6. Boost Your Day: Simple Productivity Hacks - 900 chars
7. Travel on a Budget: See the World Without Breaking the Bank - 1239 chars
8. Effective Home Workout Routi

In [15]:
import os
import json
import time
from typing import Dict, Any
from pydantic import BaseModel, ValidationError
from google import genai
from google.genai import types

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# System prompt - adapt this to your needs
system_prompt = """
You are an assistant that writes engaging blog posts. 
Create content that is informative, well-structured, and under 300 words.
Format your response as a JSON object with 'title' and 'content' fields.
"""

class BlogPost(BaseModel):
    title: str
    content: str

async def generate_post_with_gemini(user_prompt: str) -> Dict[str, Any]:
    """Generate a post based on a user-provided prompt using Google Gemini."""
    try:
        # Initialize the client
        client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
        
        # Generate response using Gemini API
        response = await client.aio.models.generate_content(
            model='gemini-2.0-flash',
            contents=user_prompt,
            config=types.GenerateContentConfig(
                response_mime_type='application/json',
                response_schema=BlogPost,
                system_instruction=types.Part.from_text(text=system_prompt),
            ),
        )
        
        # Extract the response
        print("Generated post based on user prompt.")
        content = json.loads(response.text)
        
        # Validate and parse the response using Pydantic
        blog_post = BlogPost(**content)
        
        return {
            "title": blog_post.title,
            "content": blog_post.content,
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
    except ValidationError as ve:
        print(f"Validation error: {ve}")
        return {
            "title": "Validation Error",
            "content": f"Validation Error: {str(ve)}",
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
    except Exception as e:
        print(f"Error generating post: {e}")
        return {
            "title": "Error",
            "content": f"Error: {str(e)}",
            "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
        }
user_prompt = "Write a short blog post about the benefits of meditation. Keep it under 300 words."

# Run the asynchronous function
result = asyncio.run(generate_post_with_gemini(user_prompt))

# Display the result
print("Title:", result["title"])
print("Content:", result["content"])
print("Timestamp:", result["timestamp"])

Generated post based on user prompt.
Title: Unlocking Inner Peace: The Benefits of Meditation
Content: In today's fast-paced world, finding moments of calm can feel like a luxury. However, meditation offers a simple yet powerful way to cultivate inner peace and improve overall well-being. Regular meditation practice provides numerous benefits for both your mind and body.

One of the most significant advantages is stress reduction. Meditation helps calm the nervous system, lowering cortisol levels and promoting relaxation. This can lead to reduced anxiety and a greater sense of emotional balance. Beyond stress relief, meditation enhances focus and concentration. By training your mind to stay present, you can improve your ability to concentrate on tasks and make better decisions.

Furthermore, meditation can positively impact your physical health. Studies have shown that it can lower blood pressure, improve sleep quality, and even boost your immune system. It also promotes self-awareness

In [ ]:
def generate_story_image_prompts_for_post(post_content_vn: str, campaign_knowledge: str, brand_name: str, day_num: int):
    print(Fore.CYAN + f"\nBước 10: Tạo các prompt ảnh theo mạch truyện cho bài đăng Ngày {day_num}...")
    MAX_POST_LENGTH_FOR_PROMPT = 2500
    truncated_content = post_content_vn[:MAX_POST_LENGTH_FOR_PROMPT]
    if len(post_content_vn) > MAX_POST_LENGTH_FOR_PROMPT:
         print(Fore.YELLOW + f"    (Nội dung bài viết rút gọn còn {MAX_POST_LENGTH_FOR_PROMPT} ký tự để tạo prompt ảnh)")
    prompt = (
        "Với vai trò là chuyên gia visual storytelling, hãy phân tích bài đăng tiếng Việt dưới đây.\n"
        "Dựa trên tên thương hiệu, mô tả kênh và nội dung bài đăng, tạo ra **3 đến 4 cặp prompt ảnh** minh họa các phần chính/cảm xúc quan trọng.\n\n"
        "Mỗi cặp prompt gồm:\n"
        "1. **`part` (string)**: Nhãn tiếng Việt ngắn gọn xác định phần minh họa (VD: 'Mở đầu', 'Nỗi trăn trở', 'Điểm sáng', 'Thông điệp chính', 'Lời kết nối', 'Hành động').\n"
        "2. **`english_prompt` (string)**: Prompt **tiếng Anh** chi tiết (chủ thể, hành động, bối cảnh, ánh sáng, màu sắc, mood, phong cách [cinematic, realistic, illustration, symbolic...], tỷ lệ [--ar 16:9, --ar 1:1]).\n"
        "3. **`vietnamese_explanation` (string)**: Giải thích **tiếng Việt** ngắn gọn lý do prompt phù hợp.\n\n"
        f"--- TÊN THƯƠNG HIỆU ---\n{brand_name}\n\n"
        f"--- MÔ TẢ KÊNH ---\n{campaign_knowledge}\n--- END KNOWLEDGE ---\n\n"
        f"--- BÀI ĐĂNG TIẾNG VIỆT (Ngày {day_num}) ---\n'''{truncated_content}'''\n--- END POST ---\n\n"
        "Yêu cầu Output:\n"
        "- Return ONLY a valid JSON object (no surrounding text/markdown) with a single key 'story_prompts'.\n"
        "- Value của 'story_prompts' là list chứa 3 hoặc 4 objects (ưu tiên 4 nếu đủ ý).\n"
        "- Mỗi object có 3 string keys: 'part', 'english_prompt', 'vietnamese_explanation', đều không rỗng."
    )
    response_data = gemini_text_generation(prompt, response_schema=True)

    if not isinstance(response_data, dict) or 'story_prompts' not in response_data:
        print(Fore.RED + f"    Lỗi: Định dạng JSON prompt truyện không đúng (thiếu 'story_prompts'). Data: {response_data}")
        return None
    prompts_list = response_data['story_prompts']
    MIN_PROMPTS = 2
    if not isinstance(prompts_list, list) or len(prompts_list) < MIN_PROMPTS:
        print(Fore.RED + f"    Lỗi: 'story_prompts' không phải list hoặc < {MIN_PROMPTS} phần tử (có {len(prompts_list) if isinstance(prompts_list, list) else 'N/A'}). Data: {response_data}")
        return None

    valid_prompts = []
    for i, item in enumerate(prompts_list):
        if not (isinstance(item, dict) and
                all(k in item for k in ['part', 'english_prompt', 'vietnamese_explanation']) and
                isinstance(item.get('part'), str) and item.get('part').strip() and
                isinstance(item.get('english_prompt'), str) and item.get('english_prompt').strip() and
                isinstance(item.get('vietnamese_explanation'), str) and item.get('vietnamese_explanation').strip()):
            print(Fore.YELLOW + f"    Cảnh báo: Item prompt truyện thứ {i+1} không hợp lệ, bỏ qua: {item}")
            continue
        else:
            valid_prompts.append({
                'part': item['part'].strip(),
                'english_prompt': item['english_prompt'].strip(),
                'vietnamese_explanation': item['vietnamese_explanation'].strip()
            })

    if len(valid_prompts) < MIN_PROMPTS:
         print(Fore.RED + f"    Lỗi: Sau khi lọc, chỉ còn {len(valid_prompts)} prompt truyện hợp lệ (< {MIN_PROMPTS}). Data: {response_data}")
         return None

    print(Fore.GREEN + f"    Đã tạo và xác thực {len(valid_prompts)} prompt ảnh theo mạch truyện cho Ngày {day_num} thành công.")
    return valid_prompts


In [ ]:
def generate_story_image_prompts_for_post(post_content_vn: str, campaign_knowledge: str, brand_name: str, day_num: int):
    print(f"\nBước 10: Tạo prompt ảnh cho bài đăng Ngày {day_num}...")

    truncated = post_content_vn[:2500]
    if len(post_content_vn) > 2500:
        print("    (Rút gọn bài viết xuống 2500 ký tự để tạo prompt)")

    prompt = (
        "Với vai trò là chuyên gia visual storytelling, hãy phân tích bài đăng tiếng Việt dưới đây.\n"
        "Dựa trên tên thương hiệu, mô tả kênh và nội dung bài đăng, tạo ra **3 đến 4 cặp prompt ảnh** minh họa các phần chính/cảm xúc quan trọng.\n\n"
        "Mỗi cặp prompt gồm:\n"
        "1. **`part` (string)**: Nhãn tiếng Việt ngắn gọn xác định phần minh họa (VD: 'Mở đầu', 'Nỗi trăn trở', 'Điểm sáng', 'Thông điệp chính', 'Lời kết nối', 'Hành động').\n"
        "2. **`english_prompt` (string)**: Prompt **tiếng Anh** chi tiết (chủ thể, hành động, bối cảnh, ánh sáng, màu sắc, mood, phong cách [cinematic, realistic, illustration, symbolic...], tỷ lệ [--ar 16:9, --ar 1:1]).\n"
        "3. **`vietnamese_explanation` (string)**: Giải thích **tiếng Việt** ngắn gọn lý do prompt phù hợp.\n\n"
        f"--- TÊN THƯƠNG HIỆU ---\n{brand_name}\n\n"
        f"--- MÔ TẢ KÊNH ---\n{campaign_knowledge}\n--- END KNOWLEDGE ---\n\n"
        f"--- BÀI ĐĂNG TIẾNG VIỆT (Ngày {day_num}) ---\n'''{truncated_content}'''\n--- END POST ---\n\n"
        "Yêu cầu Output:\n"
        "- Return ONLY a valid JSON object (no surrounding text/markdown) with a single key 'story_prompts'.\n"
        "- Value của 'story_prompts' là list chứa 3 hoặc 4 objects (ưu tiên 4 nếu đủ ý).\n"
        "- Mỗi object có 3 string keys: 'part', 'english_prompt', 'vietnamese_e  xplanation', đều không rỗng."
    )

    response = gemini_text_generation(prompt, response_schema=True)

    if not isinstance(response, dict) or 'story_prompts' not in response:
        print("    ❌ Lỗi: Thiếu 'story_prompts' trong JSON.")
        return None

    prompts = response['story_prompts']
    if not isinstance(prompts, list) or len(prompts) < 2:
        print(f"    ❌ Lỗi: Chỉ có {len(prompts) if isinstance(prompts, list) else 'N/A'} prompt.")
        return None

    valid = []
    for i, p in enumerate(prompts):
        if all(isinstance(p.get(k), str) and p[k].strip() for k in ['part', 'english_prompt', 'vietnamese_explanation']):
            valid.append({k: p[k].strip() for k in ['part', 'english_prompt', 'vietnamese_explanation']})
        else:
            print(f"    ⚠️ Prompt {i+1} không hợp lệ, bỏ qua.")

    if len(valid) < 2:
        print(f"    ❌ Lỗi: Chỉ còn {len(valid)} prompt hợp lệ sau khi lọc.")
        return None

    print(f"    ✅ Tạo {len(valid)} prompt ảnh thành công.")
    return valid

In [ ]:
from dotenv import load_dotenv
from google import genai
from google.genai import types
from pydantic import BaseModel
import asyncio
from typing import Dict, Any, List
import os, json

load_dotenv()

class ImagePrompt(BaseModel):
    part: str
    english_prompt: str
    vietnamese_explanation: str

class ImagePromptGenerate(BaseModel):
    story_prompts: List[ImagePrompt]

def gemini_text_image_prompt_generation(text: str):
    client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
    
    system_prompt= """ 
    Với vai trò là chuyên gia visual storytelling, hãy phân tích bài đăng tiếng Việt dưới đây.
    Dựa trên tên thương hiệu, mô tả kênh và nội dung bài đăng, tạo ra **4 cặp prompt ảnh** minh họa các phần chính/cảm xúc quan trọng.
    Mỗi cặp prompt gồm:
    1. **`part` (string)**: Nhãn tiếng Việt ngắn gọn xác định phần minh họa (VD: 'Mở đầu', 'Nỗi trăn trở', 'Điểm sáng', 'Thông điệp chính', 'Lời kết nối', 'Hành động').
    2. **`english_prompt` (string)**: Prompt **tiếng Anh** chi tiết (chủ thể, hành động, bối cảnh, ánh sáng, màu sắc, mood, phong cách [cinematic, realistic, illustration, symbolic...], tỷ lệ [--ar 16:9, --ar 1:1]).
    3. **`vietnamese_explanation` (string)**: Giải thích **tiếng Việt** ngắn gọn lý do prompt phù hợp.

    Yêu cầu Output:
    - Return ONLY a valid JSON object (no surrounding text/markdown) with a single key 'story_prompts'.
    - Value của 'story_prompts' là list chứa 4 objects (ưu tiên 4 nếu đủ ý).
    - Mỗi object có 3 string keys: 'part', 'english_prompt', 'vietnamese_explanation', đều không rỗng.
    """
    # Generate response using Gemini API (synchronous version)
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=f"Trả thông tin cho nội dung sau đây: {text}", # Tạo 5 thương hiệu cho pages với các thông tin {insight} {target_customer}. Mỗi thương hiệu phải có title và story khác nhau.
        config=types.GenerateContentConfig(
            response_mime_type='application/json',
            response_schema=ImagePromptGenerate,
            system_instruction=types.Part.from_text(text=system_prompt),
        ),
    )
    
    # Extract the response
    print("Generated 5 themes based on user prompt.")
    content = json.loads(response.text)
    
    # Validate and parse the response using Pydantic
    prompts = ImagePromptGenerate(**content)
    
    return [(prompt.part, prompt.english_prompt, prompt.vietnamese_explanation) for prompt  in prompts.story_prompts]


In [ ]:
gemini_text_image_prompt_generation(""" 
📝 Bài Đăng Ngày 12 - Thế Hệ Di Sản

Ngày 12: Xây Dựng Bản Lĩnh - Trở Thành Phiên Bản Tốt Nhất Của Chính Mình
Quý ông trong cộng đồng Thế Hệ Di Sản,
Hôm nay, chúng ta cùng bàn về một chủ đề cốt lõi: bản lĩnh và lòng tự trọng. Trong thời đại biến động này, giữ vững niềm tin vào bản thân, đặc biệt với đàn ông, là một thử thách không hề nhỏ.  Nhiều người trong chúng ta, dù ở độ tuổi nào, đều từng cảm thấy lạc lõng, thiếu định hướng, thậm chí nghi ngờ chính mình. Áp lực công việc, gia đình, và xã hội… dễ dàng bào mòn sự tự tin.
Nhưng hãy nhớ: Bản lĩnh là nền tảng của thành công. Hãy tin tưởng vào chính mình!  Đây không chỉ là lời động viên, mà là chân lý cần được sống trọn vẹn.
Vậy làm thế nào để xây dựng và củng cố bản lĩnh? Đó không phải là đích đến mà là một hành trình đòi hỏi sự nỗ lực không ngừng, sự kiên trì, và nhất là sự tự nhận thức.
Đầu tiên, hãy xác định giá trị cốt lõi của bản thân.  Bạn là ai? Bạn muốn cống hiến gì cho cuộc đời? Hiểu rõ bản thân là bước đệm vững chắc cho lòng tự trọng.  
Thứ hai, hãy đặt mục tiêu nhỏ, khả thi và chinh phục từng bước một. Mỗi thành công nhỏ sẽ là viên gạch xây nên tòa lâu đài bản lĩnh.  Hãy tưởng tượng về một người đàn ông bạn ngưỡng mộ. Anh ấy đã làm gì để đạt được thành tựu như ngày hôm nay? Hãy học hỏi từ những tấm gương đó.
Ví dụ: Nếu bạn muốn cải thiện sức khỏe, hãy bắt đầu bằng việc tập thể dục 30 phút mỗi ngày.  Nếu bạn muốn thăng tiến trong sự nghiệp, hãy đặt mục tiêu hoàn thành một dự án quan trọng.  Những thành công nhỏ này sẽ tích lũy lại, tạo nên sự tự tin mạnh mẽ.
Thứ ba, hãy học cách yêu thương và chấp nhận bản thân, cả ưu điểm và khuyết điểm.  Sai lầm là điều tất yếu. Hãy xem đó là bài học quý giá để trưởng thành.  Một người đàn ông đích thực không sợ thất bại mà biết đứng dậy sau mỗi vấp ngã.
Cuối cùng, hãy kết nối với cộng đồng.  Chia sẻ với những người cùng chí hướng, học hỏi từ kinh nghiệm của họ sẽ giúp bạn vững vàng hơn trên con đường hoàn thiện bản thân. Thế Hệ Di Sản chính là cộng đồng mà bạn có thể tìm thấy sự hỗ trợ và động lực đó.  
Hành trình rèn luyện bản lĩnh là một cuộc chiến thầm lặng đòi hỏi sự dũng cảm và kiên trì.  Nhưng hãy nhớ, bạn xứng đáng với hạnh phúc và thành công. 💪
Quý ông có bí quyết nào để giữ vững bản lĩnh trong cuộc sống? Hãy chia sẻ để cùng nhau xây dựng một cộng đồng mạnh mẽ! 👇 #TheHeDiSan #BanLĩnh #NamTính #ThànhCông
""")

Generated 5 themes based on user prompt.


[('Mở đầu',
  'A cinematic shot of a lone man silhouetted against a stormy sky, symbolizing inner turmoil and the challenges of maintaining self-belief in a turbulent world. Moody lighting, desaturated colors, --ar 16:9',
  'Hình ảnh người đàn ông đơn độc đối diện với bầu trời giông bão thể hiện sự lạc lõng và thử thách trong việc giữ vững niềm tin vào bản thân.'),
 ('Giá trị cốt lõi',
  'An illustration of a man standing at a crossroads, contemplating different paths, with symbolic representations of values like integrity, kindness, and resilience glowing softly in the background. Warm, inviting colors, illustration style, --ar 1:1',
  'Hình ảnh người đàn ông đứng trước ngã rẽ cuộc đời, suy ngẫm về các giá trị cốt lõi như sự chính trực, lòng tốt và khả năng phục hồi.'),
 ('Hành động nhỏ',
  'A realistic image of a man diligently working on a small project, with sunlight streaming through the window, highlighting the sense of accomplishment and progress. Realistic style, warm lighting,

In [ ]:
import requests
import os
import asyncio
import requests
from google.cloud import storage
from dotenv import load_dotenv
import uuid
from datetime import datetime
from PIL import Image
from io import BytesIO

load_dotenv()
url = "https://api.ideogram.ai/generate"
headers = {
    "Api-Key": os.getenv("IDEO_API_KEY"),
    "Content-Type": "application/json"
}
payload = {
    "image_request": {
        "prompt": "A serene tropical beach scene with palm trees and a lighthouse at sunset",
        "aspect_ratio": "ASPECT_10_16",
        "model": "V_2",
        "magic_prompt_option": "AUTO"
    }
}

response = requests.post(url, json=payload, headers=headers)
if response.status_code == 200:
    data = response.json()
    print("Image URL:", data["data"][0]["url"])
else:
    print("Error:", response.status_code, response.text)


Error: 400 {"type": "about:blank", "title": "Bad Request", "detail": "'9:16' is not one of ['ASPECT_10_16', 'ASPECT_16_10', 'ASPECT_9_16', 'ASPECT_16_9', 'ASPECT_3_2', 'ASPECT_2_3', 'ASPECT_4_3', 'ASPECT_3_4', 'ASPECT_1_1', 'ASPECT_1_3', 'ASPECT_3_1'] - 'image_request.aspect_ratio'", "status": 400}


In [4]:
import json

# Path to your original service account file
input_path = "service_account.json"  # or wherever you saved it

# Load the JSON
with open(input_path, "r") as f:
    data = json.load(f)

# Escape the private_key field (main issue with Render envs)
if "private_key" in data:
    data["private_key"] = data["private_key"].replace("\n", "\\n")

# Convert full JSON to single-line safe string
render_safe_json = json.dumps(data)

# Print it out for you to copy-paste into Render
# print("\n✅ COPY THIS INTO RENDER SECRET FILE:\n")
# print(render_safe_json)

In [1]:
from together import Together
client = Together()

response = client.images.generate(
    prompt="a flying cat",
    model="black-forest-labs/FLUX.1-schnell",
    steps=4,
    disable_safety_checker=True,
)

print(response.data[0].url)

APIError: Error code: 500 - {"message": "Internal server error", "type_": "server_error"}

In [2]:
from together import Together

client = Together()
response = client.images.generate(
    prompt="Cats eating popcorn",
    model="black-forest-labs/FLUX.1-dev",
    steps=10,
    n=4
)
print(response.data[0].b64_json)

APIError: Error code: 500 - {"message": "Internal server error", "type_": "server_error"}

In [10]:
import os, asyncio
from together import AsyncTogether
import nest_asyncio
nest_asyncio.apply()

async_client = AsyncTogether(api_key=os.environ.get("TOGETHER_API_KEY"))
messages = [
    "What are the top things to do in San Francisco?",
    "What country is Paris in?",
]

async def async_chat_completion(messages):
    async_client = AsyncTogether(api_key=os.environ.get("TOGETHER_API_KEY"))
    tasks = [
        async_client.chat.completions.create(
            model="meta-llama/Llama-3-70b-chat-hf",
            messages=[{"role": "user", "content": message}],
        )
        for message in messages
    ]
    responses = await asyncio.gather(*tasks)

    for response in responses:
        print(response.choices[0].message.content)

asyncio.run(async_chat_completion(messages))


San Francisco, the City by the Bay, is a popular tourist destination known for its iconic landmarks, vibrant neighborhoods, and stunning natural beauty. Here are the top things to do in San Francisco:

1. **Golden Gate Bridge**: Take a walk or bike ride across the iconic suspension bridge, offering breathtaking views of the San Francisco Bay and the city skyline.
2. **Alcatraz Island**: Visit the former maximum-security prison turned national park, featuring a guided tour and stunning views of the Bay.
3. **Fisherman's Wharf**: Explore the bustling waterfront district, known for its seafood restaurants, street performers, and souvenir shops.
4. **Chinatown**: Wander through the vibrant and colorful streets of San Francisco's Chinatown, one of the largest and oldest in the United States.
5. **Union Square**: Shop and dine in this bustling square, surrounded by iconic department stores, art galleries, and theaters.
6. **Lombard Street**: Take a stroll down the "crookedest street in the w

In [13]:
# List of prompts for image generation
prompts = [
    "A futuristic cityscape at dusk",
    "A serene mountain landscape in autumn",
]

# Asynchronous function to generate images for each prompt
async def async_image_generation(prompts):
    tasks = [
        async_client.images.generate(
            prompt=prompt,
            model="black-forest-labs/FLUX.1-schnell",
            width=576,   # 9:16 aspect ratio
            height=1024,
            steps=4,
            n=1,
        )
        for prompt in prompts
    ]
    responses = await asyncio.gather(*tasks)

    # Print the URL of each generated image
    for response in responses:
        print(response.data[0].url)

# Run the asynchronous image generation function
asyncio.run(async_image_generation(prompts))

https://api.together.ai/imgproxy/drNN5ZL8IcVzm1jzMqp31hg6jbpyVmo5AmdxUnm2I4Q/format:jpeg/aHR0cHM6Ly90b2dldGhlci1haS1iZmwtaW1hZ2VzLXByb2QuczMudXMtd2VzdC0yLmFtYXpvbmF3cy5jb20vaW1hZ2VzLzM0NmU0MGQ5NGI0M2JlZjg3YmUzYjdlZDNhYTgyZjFiMmZlMzUwYWE5MTUwZWEwYzg2MDIwNGZlNzhjZjVmOTk_WC1BbXotQWxnb3JpdGhtPUFXUzQtSE1BQy1TSEEyNTYmWC1BbXotQ29udGVudC1TaGEyNTY9VU5TSUdORUQtUEFZTE9BRCZYLUFtei1DcmVkZW50aWFsPUFTSUFZV1pXNEhWQ0RYUFNRQlNRJTJGMjAyNTA0MTklMkZ1cy13ZXN0LTIlMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjUwNDE5VDEzNTgzNlomWC1BbXotRXhwaXJlcz0zNjAwJlgtQW16LVNlY3VyaXR5LVRva2VuPUlRb0piM0pwWjJsdVgyVmpFQVlhQ1hWekxYZGxjM1F0TWlKSE1FVUNJRnBRb1ZnRnZpYUtBdHZvRVc5ZENrYWI4RnRKVENhR2NiTlBteXcxb2RHMUFpRUF6b1NrNjYyeGZOb2IlMkZDZEJIWGFyblhTaWprWk5hYnM4R0FFTHdDR3psbjRxbVFVSWolMkYlMkYlMkYlMkYlMkYlMkYlMkYlMkYlMkYlMkYlMkZBUkFBR2d3MU9UZzNNall4TmpNM09EQWlETWt6bCUyQkNHUFpaN3RIN3VHU3J0QkZEUXJKdDJxVWU4Rk1QNGUwS2N0WFJtTHlEdlY5R0FCY2FuR1dERjFvdG10MElsR1VpaTlIWHc3WjljaDdSY2lTN3lhMVQ2WjRHZnNYQktXVWJ4eGpsNmc5b2RwYmtCVWZranVVSWJ6dUxVcmt

In [15]:
import os
from together import Together

client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

response = client.images.generate(
    prompt="space robots",
    model="black-forest-labs/FLUX.1-schnell",
    steps=10,
    n=2,
)
print(response.data[0].b64_json)

None
